Experimento 2 - KNN Imputer, 3sigma outlier detection, min max normalization, MLP classifier

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import seaborn.objects as so
!pip install --quiet ucimlrepo
from ucimlrepo import fetch_ucirepo

In [2]:
beans = fetch_ucirepo(id=602)
df = beans.data.features
targets = beans.data.targets

In [3]:
cols = ['Area', 'Perimeter', 'MajorAxisLength', 'MinorAxisLength', 'AspectRatio', 'Eccentricity', 'ConvexArea', 'EquivDiameter', 'Extent', 'Solidity', 'Roundness', 'Compactness', 'ShapeFactor1', 'ShapeFactor2', 'ShapeFactor3', 'ShapeFactor4']

In [4]:
import random

#Introducing Missing values (5%)
x = df.copy()

for index, i in enumerate(df):
  for jndex, j in enumerate(df[i]):
    if random.randint(0,100) < 5:
      x[i][jndex] = np.NaN

df = x

<ipython-input-4-fbb7389abead>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i][jndex] = np.NaN


In [5]:
df.isna().sum()

Area               704
Perimeter          670
MajorAxisLength    691
MinorAxisLength    720
AspectRatio        655
Eccentricity       708
ConvexArea         697
EquivDiameter      645
Extent             637
Solidity           652
Roundness          651
Compactness        640
ShapeFactor1       660
ShapeFactor2       660
ShapeFactor3       693
ShapeFactor4       730
dtype: int64

In [6]:
df

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
0,28395.0,610.291,208.178117,173.888747,1.197191,0.549812,28715.0,190.141097,0.763923,NaN,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724
1,28734.0,638.018,200.524796,182.734419,1.097356,NaN,29172.0,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430
2,29380.0,624.110,212.826130,175.931143,1.209713,0.562727,29690.0,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066
3,30008.0,645.884,210.557999,182.516516,NaN,0.498616,30724.0,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199
4,30140.0,620.134,201.847882,190.279279,1.060798,0.333680,30417.0,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13606,42097.0,759.696,NaN,185.944705,1.552728,0.765002,42508.0,231.515799,0.714574,NaN,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385
13607,42101.0,757.499,281.576392,190.713136,1.476439,0.735702,42494.0,231.526798,0.799943,NaN,0.922015,0.822252,0.006688,0.001886,0.676099,0.998219
13608,42139.0,NaN,281.539928,191.187979,1.472582,0.734065,42569.0,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767
13609,NaN,763.779,283.382636,190.275731,1.489326,0.741055,NaN,231.653247,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222


In [7]:
#Imputing Missing Values with KNN Imputer
from sklearn.impute import KNNImputer as knni

imputer = knni(n_neighbors=5)
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns = df.columns)
df_imputed

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
0,28395.0,610.2910,208.178117,173.888747,1.197191,0.549812,28715.0,190.141097,0.763923,0.988343,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724
1,28734.0,638.0180,200.524796,182.734419,1.097356,0.651878,29172.0,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430
2,29380.0,624.1100,212.826130,175.931143,1.209713,0.562727,29690.0,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066
3,30008.0,645.8840,210.557999,182.516516,1.376209,0.498616,30724.0,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199
4,30140.0,620.1340,201.847882,190.279279,1.060798,0.333680,30417.0,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13606,42097.0,759.6960,284.383002,185.944705,1.552728,0.765002,42508.0,231.515799,0.714574,0.989052,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385
13607,42101.0,757.4990,281.576392,190.713136,1.476439,0.735702,42494.0,231.526798,0.799943,0.988674,0.922015,0.822252,0.006688,0.001886,0.676099,0.998219
13608,42139.0,764.5378,281.539928,191.187979,1.472582,0.734065,42569.0,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767
13609,42158.0,763.7790,283.382636,190.275731,1.489326,0.741055,42651.8,231.653247,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222


In [8]:
#adding the labels
df_imputed['Class'] = targets
df_pre_missing_values = df.copy()
print(df_imputed.groupby('Class').agg(['count']))
df_imputed['Class'] = df_imputed['Class'].transform(lambda x: 0 if x == 'BARBUNYA' else (1 if x == 'BOMBAY' else (2 if x == 'CALI' else (3 if x == 'DERMASON' else (4 if x == 'HOROZ' else (5 if x == 'SEKER' else 6))))))

          Area Perimeter MajorAxisLength MinorAxisLength AspectRatio  \
         count     count           count           count       count   
Class                                                                  
BARBUNYA  1322      1322            1322            1322        1322   
BOMBAY     522       522             522             522         522   
CALI      1630      1630            1630            1630        1630   
DERMASON  3546      3546            3546            3546        3546   
HOROZ     1928      1928            1928            1928        1928   
SEKER     2027      2027            2027            2027        2027   
SIRA      2636      2636            2636            2636        2636   

         Eccentricity ConvexArea EquivDiameter Extent Solidity Roundness  \
                count      count         count  count    count     count   
Class                                                                      
BARBUNYA         1322       1322          1322   13

In [9]:
#Outlier Removal with 3sigma
from scipy import stats


print(f'Pre Outlier Shape: {df_imputed.shape}')
df_no_outliers = df_imputed.copy()

print(df_no_outliers['Class'].unique())
for i in df_no_outliers['Class'].unique():
    class_unique = df_no_outliers[df_no_outliers['Class'] == i]
    for feature in class_unique:
      upper = class_unique[feature].mean() + (3 * class_unique[feature].std())
      lower = class_unique[feature].mean() - (3 * class_unique[feature].std())
      excluded = pd.Series(class_unique[class_unique[feature] < lower].index)
      #print(excluded.values)
      df_no_outliers.drop(excluded.values, inplace = True, errors='ignore')

print(df_no_outliers.groupby('Class').count())
print(f'Pos Outlier Shape: {df_no_outliers.shape}')


Pre Outlier Shape: (13611, 17)
[5 0 1 2 4 6 3]
       Area  Perimeter  MajorAxisLength  MinorAxisLength  AspectRatio  \
Class                                                                   
0      1264       1264             1264             1264         1264   
1       491        491              491              491          491   
2      1578       1578             1578             1578         1578   
3      3406       3406             3406             3406         3406   
4      1804       1804             1804             1804         1804   
5      1913       1913             1913             1913         1913   
6      2534       2534             2534             2534         2534   

       Eccentricity  ConvexArea  EquivDiameter  Extent  Solidity  Roundness  \
Class                                                                         
0              1264        1264           1264    1264      1264       1264   
1               491         491            491     491    

In [10]:
df_no_outliers['Class'] = df_no_outliers['Class'].transform(lambda x: 'BARBUNYA' if x == 0 else ('BOMBAY' if x == 1 else ('CALI' if x == 2 else ('DERMASON' if x == 3 else ('HOROZ' if x == 4 else ('SEKER' if x == 5 else 'SIRA'))))))
df_no_outliers = df_no_outliers.reset_index()
print(df_no_outliers.groupby('Class').agg(['count']))

         index  Area Perimeter MajorAxisLength MinorAxisLength AspectRatio  \
         count count     count           count           count       count   
Class                                                                        
BARBUNYA  1264  1264      1264            1264            1264        1264   
BOMBAY     491   491       491             491             491         491   
CALI      1578  1578      1578            1578            1578        1578   
DERMASON  3406  3406      3406            3406            3406        3406   
HOROZ     1804  1804      1804            1804            1804        1804   
SEKER     1913  1913      1913            1913            1913        1913   
SIRA      2534  2534      2534            2534            2534        2534   

         Eccentricity ConvexArea EquivDiameter Extent Solidity Roundness  \
                count      count         count  count    count     count   
Class                                                              

In [11]:
df_no_outliers = df_no_outliers.drop(['index'], axis='columns')
df_no_outliers

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395.0,610.2910,208.178117,173.888747,1.197191,0.549812,28715.0,190.141097,0.763923,0.988343,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734.0,638.0180,200.524796,182.734419,1.097356,0.651878,29172.0,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29380.0,624.1100,212.826130,175.931143,1.209713,0.562727,29690.0,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
3,34791.4,676.9420,212.560556,181.510182,1.171067,0.520401,30600.0,196.347702,0.775688,0.989510,0.943852,0.923726,0.007020,0.002969,0.853270,0.999236,SEKER
4,30477.0,670.0330,211.050155,184.039050,1.146768,0.489478,30970.0,196.988633,0.762402,0.984081,0.853080,0.933374,0.006925,0.003242,0.871186,0.999049,SEKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12985,42097.0,759.6960,284.383002,185.944705,1.552728,0.765002,42508.0,231.515799,0.714574,0.989052,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385,DERMASON
12986,42101.0,757.4990,281.576392,190.713136,1.476439,0.735702,42494.0,231.526798,0.799943,0.988674,0.922015,0.822252,0.006688,0.001886,0.676099,0.998219,DERMASON
12987,42139.0,764.5378,281.539928,191.187979,1.472582,0.734065,42569.0,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767,DERMASON
12988,42158.0,763.7790,283.382636,190.275731,1.489326,0.741055,42651.8,231.653247,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222,DERMASON


In [12]:
#Normalization with min max
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit_transform(df_no_outliers[cols])
df_scaled = pd.DataFrame(scaler.transform(df_no_outliers[cols]), columns = cols)

df_scaled['Class'] = df_no_outliers['Class']
df_scaled

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,0.034791,0.060789,0.041240,0.134028,0.087594,0.325651,0.033564,0.071030,0.671024,0.762977,0.902127,0.848057,0.639626,0.861196,0.822198,0.961646,SEKER
1,0.036278,0.080647,0.027395,0.160782,0.013725,0.515989,0.035495,0.073829,0.735504,0.637342,0.689368,0.973978,0.590062,1.000000,0.968892,0.950484,SEKER
2,0.039112,0.070686,0.049649,0.140205,0.096859,0.349735,0.037684,0.079116,0.716671,0.808464,0.871626,0.833807,0.627322,0.828005,0.805998,0.974660,SEKER
3,0.062851,0.108524,0.049169,0.157079,0.068264,0.270803,0.041529,0.086379,0.708871,0.806632,0.859645,0.880291,0.595878,0.801906,0.859144,0.981111,SEKER
4,0.043924,0.103576,0.046436,0.164728,0.050285,0.213136,0.043093,0.087964,0.666131,0.603507,0.587612,0.910285,0.582510,0.892778,0.893896,0.973999,SEKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12985,0.094898,0.167792,0.179102,0.170492,0.350663,0.726947,0.091848,0.173345,0.512286,0.789503,0.777984,0.501434,0.573181,0.395047,0.451264,0.948770,DERMASON
12986,0.094916,0.166218,0.174025,0.184915,0.294215,0.672307,0.091788,0.173373,0.786890,0.775359,0.794204,0.564816,0.549250,0.440635,0.515489,0.942435,DERMASON
12987,0.095082,0.171259,0.173959,0.186351,0.291361,0.669254,0.092105,0.173631,0.561689,0.821186,0.783441,0.566301,0.548281,0.441448,0.517013,0.887246,DERMASON
12988,0.095166,0.170716,0.177293,0.183592,0.303751,0.682289,0.092455,0.173685,0.482741,0.743124,0.751921,0.549910,0.554244,0.429363,0.500239,0.828503,DERMASON


In [19]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate


x_train = df_scaled.iloc[:, 0:16]
y_train = df_scaled.iloc[:, 16]
classifier = MLPClassifier(activation='logistic', solver='adam', alpha=1e-5, hidden_layer_sizes=(12, 3), random_state=1, verbose=True, learning_rate_init=0.3, tol=1e-3, max_iter=500)
scoring = {'acc' : 'accuracy',
           'prec' : 'precision_macro',
           'recall' : 'recall_macro',
           'f1' : 'f1_macro'}


y_pred = cross_validate(classifier, x_train, y_train, cv=10, scoring=scoring, return_train_score=True)
print('Experimento 2')
print('Acurácia Média: ' + '%.2f' % (np.mean(y_pred['test_acc'])*100) + '%')
print('Precisão Média: ' + '%.2f' % (np.mean(y_pred['test_prec'])*100) + '%')
print('Revocação Média: ' + '%.2f' % (np.mean(y_pred['test_recall'])*100) + '%')
print('F1-Score Médio: ' + '%.2f' % (np.mean(y_pred['test_f1'])*100) + '%')


Iteration 1, loss = 1.33501547
Iteration 2, loss = 0.76768368
Iteration 3, loss = 0.61472396
Iteration 4, loss = 0.49463522
Iteration 5, loss = 0.33997516
Iteration 6, loss = 0.29729614
Iteration 7, loss = 0.29967362
Iteration 8, loss = 0.29802048
Iteration 9, loss = 0.28131126
Iteration 10, loss = 0.26304699
Iteration 11, loss = 0.25158493
Iteration 12, loss = 0.25983768
Iteration 13, loss = 0.27483666
Iteration 14, loss = 0.25400421
Iteration 15, loss = 0.24099307
Iteration 16, loss = 0.25208445
Iteration 17, loss = 0.22685503
Iteration 18, loss = 0.21773778
Iteration 19, loss = 0.31067527
Iteration 20, loss = 0.21689533
Iteration 21, loss = 0.21274761
Iteration 22, loss = 0.26527680
Iteration 23, loss = 0.25756139
Iteration 24, loss = 0.24763976
Iteration 25, loss = 0.21192927
Iteration 26, loss = 0.21691073
Iteration 27, loss = 0.20703414
Iteration 28, loss = 0.24576414
Iteration 29, loss = 0.20990921
Iteration 30, loss = 0.23143230
Iteration 31, loss = 0.22018527
Iteration 32, los

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Iteration 1, loss = 1.30318508
Iteration 2, loss = 0.96124957
Iteration 3, loss = 0.78757211
Iteration 4, loss = 0.64658623
Iteration 5, loss = 0.54423780
Iteration 6, loss = 0.50322023
Iteration 7, loss = 0.51158154
Iteration 8, loss = 0.49146153
Iteration 9, loss = 0.48700588
Iteration 10, loss = 0.48848144
Iteration 11, loss = 0.48553750
Iteration 12, loss = 0.47859912
Iteration 13, loss = 0.46585926
Iteration 14, loss = 0.45957794
Iteration 15, loss = 0.53047267
Iteration 16, loss = 0.47215975
Iteration 17, loss = 0.44452334
Iteration 18, loss = 0.47235722
Iteration 19, loss = 0.51128064
Iteration 20, loss = 0.48392878
Iteration 21, loss = 0.43581749
Iteration 22, loss = 0.44291441
Iteration 23, loss = 0.44740937
Iteration 24, loss = 0.44390707
Iteration 25, loss = 0.44115559
Iteration 26, loss = 0.40663834
Iteration 27, loss = 0.40019775
Iteration 28, loss = 0.38488200
Iteration 29, loss = 0.36824788
Iteration 30, loss = 0.35350256
Iteration 31, loss = 0.34567706
Iteration 32, los